# Tutorial 4 : The Lattice Boltzmann Method in ESPResSo - Part 4

### 6 Poiseuille flow in ESPResSo

Poiseuille flow is the flow through a pipe or (in our case) a slit
under a homogenous force density, e.g. gravity. In the limit of small Reynolds
numbers, the flow can be described with the Stokes equation. 
We assume the slit being infinitely extended in $y$ and $z$ 
direction and a force density $f_y$ on the fluid 
in $y$ direction. No slip-boundary conditions  (i.e. $\vec{u}=0$)
are located at $x = \pm h/2$.
Assuming invariance in $y$ and $z$ direction and a steady state, 
the Stokes equation is simplified to:

\begin{equation}
  \eta \partial_x^2 u_y = f_y
\end{equation}

where $f_y$ denotes the force density and $\eta$ the dynamic viscosity.
This can be integrated twice and the integration constants are chosen
so that $u_y=0$ at $x = \pm h/2$ to obtain the solution to the
planar Poiseuille flow [8]:

\begin{equation}
  u_y(x) = \frac{f_y}{2\eta} \left(h^2/4-x^2\right)
\end{equation}

We will simulate a planar Poiseuille flow using a square box, two walls
with normal vectors $\left(\pm 1, 0, 0 \right)$, and an external force
applied to every node.

Use the data to fit a parabolic function. Can you confirm the analytic solution?

In [ ]:
import espressomd
espressomd.assert_features(['CUDA', 'LB_BOUNDARIES_GPU'])
from espressomd import System, lb, shapes, lbboundaries
import numpy as np

# System setup
box_l = 16.0
agrid = 0.5
visc = 1.0
force_density = 0.001
system = System(box_l=[box_l, box_l, box_l])
system.set_random_state_PRNG()
np.random.seed(seed=system.seed)
system.time_step = 0.01
system.cell_system.skin = 0.2

lbf = lb.LBFluidGPU(agrid=agrid, dens=1, visc=visc, tau=0.01,
                    ext_force_density=[0, force_density, 0])
system.actors.add(lbf)

# Setup boundaries
wall_offset = 1.5
walls = [lbboundaries.LBBoundary() for k in range(2)]
walls[0].set_params(shape=shapes.Wall(normal=[1, 0, 0], dist=wall_offset))
walls[1].set_params(shape=shapes.Wall(normal=[-1, 0, 0], dist=-(box_l - wall_offset)))

for wall in walls:
    system.lbboundaries.add(wall)

# Iterate until the flow profile converges (5000 LB updates)
system.integrator.run(5000)

# Extract fluid velocity along the x-axis
node_v_list = []
for i in range(int(round(box_l / float(agrid)))):
    node_v_list.append(lbf[i, 0, 0].velocity[1])

# Write to file
with open("lb_fluid_velocity.dat", "w") as f:
    for line in node_v_list:
        f.write(str(line) + "\n")

<figure>
<img src='figures/poiseuille.png', style="width: 500px;"/>
<center>
<figcaption>Poiseuille flow in a slit geometry.<br>
The solution is available at <tt>/doc/tutorials/04-lattice_boltzmann/scripts/part4_plot.py</tt> </figcaption>
</center>
</figure>

## References

[1] S Succi. *The lattice Boltzmann equation for fluid dynamics and beyond. *Clarendon Press, Oxford, 2001.  
[2] B. Dünweg and A. J. C. Ladd. *Advanced Computer Simulation Approaches for Soft Matter Sciences III*, chapter II, pages 89–166. Springer, 2009.  
[3] B. Dünweg, U. Schiller, and A.J.C. Ladd. Statistical mechanics of the fluctuating lattice-boltzmann equation. *Phys. Rev. E*, 76:36704, 2007.  
[4] P. G. de Gennes. *Scaling Concepts in Polymer Physics*. Cornell University Press, Ithaca, NY, 1979.  
[5] M. Doi. *Introduction to Polymer Physics.* Clarendon Press, Oxford, 1996.  
[6] Michael Rubinstein and Ralph H. Colby. *Polymer Physics.* Oxford University Press, Oxford, UK, 2003.  
[7] Daan Frenkel and Berend Smit. *Understanding Molecular Simulation.* Academic Press, San Diego, second edition, 2002.  
[8] W. E. Langlois and M. O. Deville. *Exact Solutions to the Equations of Viscous Flow*. In: *Slow Viscous Flow*, Springer, Cham, 2014.